In [1]:
import os
directory = os.getcwd()
sample_folder = '\\res'
res_folder = '\\res\\trn'
os.chdir(directory+res_folder)

In [49]:
import numpy as np
import pandas as pd
test = os.listdir()
avl_labels_og = [] # avaliable labels in the 
for i in test:
    if '_temp_flow' in i:
        prefix = i[:-14]
    elif '_control_signal' in i:
        prefix = i[:-19]
    elif '_energy' in i:
        prefix = i[:-11]
    else:
        continue
    avl_labels_og.append(prefix)
print(len(avl_labels_og))        
avl_labels_og = np.array(avl_labels_og)
avl_labels_og = np.unique(avl_labels_og)
print(len(avl_labels_og))
avl_labels_og

1480
496


array(['0', '1', '10', '100', '101', '102', '103', '104', '105', '106',
       '107', '108', '109', '11', '110', '111', '112', '113', '114',
       '115', '116', '117', '118', '119', '12', '120', '121', '122',
       '123', '124', '125', '126', '127', '128', '129', '13', '130',
       '131', '132', '133', '134', '135', '136', '137', '138', '139',
       '14', '140', '141', '142', '143', '144', '145', '146', '147',
       '148', '149', '15', '150', '151', '152', '153', '154', '155',
       '156', '157', '158', '159', '16', '160', '161', '162', '163',
       '164', '165', '166_cp', '167_cp', '168_cp', '169_cp', '17',
       '170_cp', '171_cp', '172_cp', '173_cp', '174_cp', '175_cp',
       '176_cp', '177_cp', '178_cp', '179_cp', '18', '180_cp', '181',
       '182_cp', '183_cp', '184_cp', '185_cp', '186_cp', '187_cp',
       '188_cp', '189_cp', '19', '190_cp', '191_cp', '192_cp', '193_cp',
       '194_cp', '195_cp', '196_cp', '197_cp', '198_cp', '199_cp', '2',
       '20', '200_cp', '201_

In [3]:
import re
avl_labels = np.array([re.search(r'\d+', item).group() for item in avl_labels_og if re.search(r'\d+', item)])
len(avl_labels)

440

In [4]:
os.chdir(directory + sample_folder)
#previous = np.arange(86)
#previous = previous.astype(str)
#previous = pd.DataFrame(previous, columns=['label'])

previous = pd.concat([pd.read_csv('morris_sample_st.csv'),
                     pd.read_csv('morris_sample_st2.csv'),
                     pd.read_csv('morris_sample_pvt.csv'),
                     pd.read_csv('morris_sample_pvt2.csv'),
                     pd.read_csv('morris_sample_cp.csv'),
                     pd.read_csv('morris_sample_cp2.csv'),
                     pd.read_csv('morris_sample_batt1.csv'),
                     pd.read_csv('morris_sample_cp.csv')],
                    ignore_index=True)
previous = previous.drop_duplicates(ignore_index=True)
previous

,volume,coll_area,flow_rate,design_case,r_level
0,0.40,8.0,200,ST,r0
1,0.20,8.0,200,ST,r0
2,0.20,20.0,200,ST,r0
3,0.20,20.0,100,ST,r0
4,0.10,8.0,100,ST,r0
...,...,...,...,...,...
257,0.25,16.0,200,PVT_9,r0
258,0.25,20.0,200,PVT_9,r0
259,0.25,20.0,200,PVT_6,r0
260,0.20,8.0,50,PVT_6,r0


In [5]:
df1 = pd.read_csv('morris_sample_st.csv')
df2 = pd.read_csv('morris_sample_st2.csv')
df3 = pd.read_csv('morris_sample_pvt.csv')
df4 = pd.read_csv('morris_sample_pvt2.csv')
dfnew = pd.concat([df1,df2,df3,df4,
                   pd.read_csv('morris_sample_cp.csv'),
                   pd.read_csv('morris_sample_cp2.csv'),
                   pd.read_csv('morris_sample_batt1.csv'),
                   pd.read_csv('morris_sample_cp3.csv'),
                   pd.read_csv('morris_sample_pvt4.csv'),
                   pd.read_csv('morris_sample_st4.csv'),
                  pd.read_csv(res_folder+'lhs_sample_1.csv')])

dfnew.index = np.arange(len(dfnew))
dfnew = dfnew.drop_duplicates()
dfnew.index = np.arange(len(dfnew))
current_list = pd.merge(dfnew, previous, how='outer', indicator=True)
current_list = current_list[current_list['_merge'] == 'left_only']
current_list.drop(columns=['_merge'], inplace=True)
current_list.index = np.arange(181, 181+len(current_list))
current_list

FileNotFoundError: [Errno 2] No such file or directory: 'morris_sample_st.csv'

In [55]:
all_files = os.listdir('..')
sample_files = []
for file in all_files:
    if 'sample' in file:
        sample_files.append(file)
sample_files

['lhs_sample_1.csv',
 'morris_sample_batt1.csv',
 'morris_sample_cp.csv',
 'morris_sample_cp2.csv',
 'morris_sample_cp3.csv',
 'morris_sample_pvt.csv',
 'morris_sample_pvt2.csv',
 'morris_sample_pvt3.csv',
 'morris_sample_pvt4.csv',
 'morris_sample_st.csv',
 'morris_sample_st2.csv',
 'morris_sample_st3.csv',
 'morris_sample_st4.csv']

In [7]:
total_labels = pd.concat([previous,current_list], ignore_index=False).index
total_labels=total_labels.astype(str)
total_labels=total_labels.values
len(total_labels)

290

In [8]:
missing = set(total_labels)-set(avl_labels)
missing

set()

In [50]:
# check length of all files and flag the onces that are incomplete
os.chdir(directory+res_folder)
import pandas as pd
repeat = pd.DataFrame()
inp = pd.read_csv('list_of_inputs.csv', header=0,index_col=0)
for i in avl_labels_og:
    df = pd.read_csv(i+'_control_signal.txt', delimiter=',',index_col=0) 
    if len(df) <87601 :
        print(i + ' len = '+str(len(df)))
        j = ''.join([char for char in i if char.isdigit()])
        repeat = repeat.append(inp.loc[int(j)])
repeat

479 len = 22161
x len = 45871


ValueError: invalid literal for int() with base 10: ''

In [4]:
repeat.to_csv('..\\missed_sims5.csv',index_label='label')

NameError: name 'dfnew' is not defined

In [42]:
slice_list = [356]
df2 = dfnew.iloc[dfnew.index.isin(slice_list)]
df2

In [24]:
sim_results = pd.read_csv('sim_results.csv')
sim_results['lab'] = sim_results['lab'].astype(int)
len(sim_results)

163

In [45]:
x = list(sim_results['lab'])
y = list(dfnew.index)
list(set(y)-set(x))

[164, 165, 150]